In [15]:
# Imports
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans

# Load IRIS data

In [16]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

# K-clustering algorithm

In [30]:
n_clusters = 3
k_means = KMeans(n_clusters = n_clusters, n_init = 50)
k_means.fit(X)

centers = k_means.cluster_centers_
print(centers)

[[5.006      3.428      1.462      0.246     ]
 [5.9016129  2.7483871  4.39354839 1.43387097]
 [6.85       3.07368421 5.74210526 2.07105263]]


# PSO Clustering

#### Parameters:

In [ ]:
n_clusters = 3 # according to the assignment
particles = 10 # according to the paper
omega  = 0.72 # according to the paper
c1 = 1.49 # according to the paper
c2 = 1.49 # according to the paper

Quality is measured according to the following three criteria (according to the paper):
- the quantization error
- the intra-cluster distances, i.e. the distance between data vectors within a cluster, where the objective is to minimize the intra-cluster distances
- the inter-clus!er distances, i.e. the distance between the centroids of the clusters, where the objective is to maximize the distance between clusters